In [2]:
import os #provides a way of interacting with the operating system. It's commonly used for tasks like file and directory manipulation.
import numpy as np
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
#ImageDataGenerator is used for data augmentation and preprocessing of images, which is essential for training deep learning models on image data.
from tensorflow.keras.models import Sequential
#Sequential is a type of Keras model that allows you to build neural networks layer by layer in a sequential manner.
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
'''Conv2D: Performs 2D spatial convolution to extract features from input images.
MaxPooling2D: Performs downsampling by retaining the maximum value in each region of the input feature maps.
Flatten: Converts spatial feature maps into a 1D vector for feeding into fully connected layers.
Dense: Represents fully connected layers for learning nonlinear relationships in the data.'''
from sklearn.model_selection import train_test_split

In [3]:

dataset_dir = r"C:\Users\penfr\OneDrive\Desktop\Intellipaat_Python\AI materials\cats_dogs_dataset" 





In [4]:
# Initialize an ImageDataGenerator object for augmenting training images
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values between [0,1]
    rotation_range=40,  # Range for random rotations
    width_shift_range=0.2,  # Range for random horizontal shifts
    height_shift_range=0.2,  # Range for random vertical shifts
    shear_range=0.2,  # Range for random shearing
    zoom_range=0.2,  # Range for random zoom
    horizontal_flip=True  # Randomly flip images horizontally
)


Parameters like rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, and horizontal_flip are specified to apply various transformations to the images. These transformations increase the diversity of the training dataset, which helps prevent overfitting and improves the model's ability to generalize to unseen data.

In [5]:
# Create a generator for training data
train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_dir, "PetImages"),  # Directory containing training images
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,  # Number of images in each batch
    class_mode='binary'  # Binary class mode for binary classification
)

Found 25000 images belonging to 2 classes.


In [4]:
# Splitting the dataset into train and test sets
train_files, test_files = train_test_split(train_generator.filepaths, test_size=0.2, random_state=42)

NameError: name 'train_generator' is not defined

In [7]:
# Model Building:
# Construct a Convolutional Neural Network using TensorFlow and Keras
model = Sequential([
    # Add a 2D convolutional layer with 32 filters, each with a 3x3 kernel size,
    # ReLU activation, and input shape of (150, 150, 3) for RGB images
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    # Convolutional layers are responsible for extracting features from input images.
    # The first convolutional layer typically learns low-level features like edges and textures.

    # Add a max pooling layer with a pool size of 2x2
    MaxPooling2D(2, 2),
    # Max pooling layers reduce the spatial dimensions of the feature maps, reducing computation
    # and providing translation invariance to some extent.

    # Add another 2D convolutional layer with 64 filters and a 3x3 kernel size, followed by ReLU activation
    Conv2D(64, (3, 3), activation='relu'),
    # Additional convolutional layers learn higher-level features by combining lower-level features.
    # ReLU activation introduces non-linearity to the network.

    # Add another max pooling layer
    MaxPooling2D(2, 2),

    # Add a third 2D convolutional layer with 128 filters and a 3x3 kernel size, followed by ReLU activation
    Conv2D(128, (3, 3), activation='relu'),

    # Add another max pooling layer
    MaxPooling2D(2, 2),

    # Add a fourth 2D convolutional layer with 128 filters and a 3x3 kernel size, followed by ReLU activation
    Conv2D(128, (3, 3), activation='relu'),

    # Add another max pooling layer
    MaxPooling2D(2, 2),

    # Flatten the 2D feature maps into a 1D vector
    Flatten(),
    # Flatten layer converts the output of convolutional layers into a format suitable for feeding
    # into fully connected layers.

    # Add a fully connected layer with 512 neurons and ReLU activation
    Dense(512, activation='relu'),
    # Dense layers, also known as fully connected layers, learn complex patterns in the data.

    # Add the output layer with a single neuron and sigmoid activation for binary classification
    Dense(1, activation='sigmoid')
    # The output layer produces the final prediction, with sigmoid activation for binary classification.
])


In [5]:
# Training:
# Compile the model and train it on the prepared dataset
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# Compiles the model with binary cross-entropy loss function, Adam optimizer,
# and accuracy metric for evaluation during training.

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs=20
)
# Fits the model to the training data generator for a specified number of epochs,
# with steps_per_epoch calculated based on the number of samples and batch size in the generator.



NameError: name 'model' is not defined

In [ ]:
# Evaluation:
# Evaluate the model's performance on the validation set during training
test_datagen = ImageDataGenerator(rescale=1./255)
# Initialize an ImageDataGenerator for preprocessing the test images.

test_generator = test_datagen.flow_from_directory(
    os.path.join(dataset_dir, "PetImages"),
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
# Create a generator for test data from the test directory, with resizing to (150, 150) pixels,
# batch size of 32, and binary class mode for binary classification.

test_loss, test_acc = model.evaluate(test_generator)
# Evaluates the model's performance on the test data generator, calculating test loss and accuracy.

print(f'Test accuracy: {test_acc}')
# Prints the test accuracy obtained from evaluating the model on the test data.



In [ ]:
# Prediction:
# Implement a system to make predictions on new images
sample_img_path = "path_to_sample_image.jpg"
# Specify the path to the sample image for prediction.

img = tf.keras.preprocessing.image.load_img(sample_img_path, target_size=(150, 150))
# Load the sample image and resize it to (150, 150) pixels.

img_array = tf.keras.preprocessing.image.img_to_array(img)
# Convert the image to a numpy array.

img_array = np.expand_dims(img_array, axis=0)
# Expand the dimensions of the image array to match the model's input shape.

img_array /= 255.
# Normalize pixel values to the range [0, 1].

prediction = model.predict(img_array)
# Make predictions on the image array using the trained model.

if prediction[0] > 0.5:
    print("Dog")
else:
    print("Cat")
# Print the predicted class based on the model's prediction probability.
